In [ ]:
import $ivy.`io.prometheus:simpleclient:0.8.1`
import $ivy.`io.prometheus:simpleclient_common:0.8.1`
import $ivy.`dev.zio::zio:1.0.0-RC18-2`

In [12]:
CollectorRegistry.defaultRegistry.clear()

In [ ]:
// This is a Java Library
import io.prometheus.client.CollectorRegistry
import io.prometheus.client.{ Counter, Histogram }
import io.prometheus.client.exporter.common.TextFormat

// Spread operator => :_*
val c = Counter
    .build()
    .name("PrometheusCounter")
    .labelNames(Array("class", "method"): _*)
    .help(s"Sample prometheus counter")
    .register()

val h = Histogram
    .build()
    .name("PrometheusHistogram")
    .labelNames(Array("class", "method"): _*)
    .linearBuckets(0.0, 0.2, 5)
    .help(s"Sample prometheus histogram")
    .register()

In [14]:
c.labels(Array("Users", "getTotal"):_*).inc()
c.labels(Array("Users", "getUsers"):_*).inc(3)

h.labels(Array("Users", "getTotal"):_*).observe(0.32)
h.labels(Array("Users", "getUsers"):_*).observe(0.45)

In [ ]:
import java.io.StringWriter

def write004(registry: CollectorRegistry) = {
    val writer = new StringWriter
    TextFormat.write004(writer, registry.metricFamilySamples)
    writer.toString
}

write004(CollectorRegistry.defaultRegistry)

In [21]:
import zio.console.putStrLn
import zio.console.Console
import zio.Runtime

val writeT004: CollectorRegistry => Task[String] = registry => Task {
    val writer = new StringWriter
    TextFormat.write004(writer, registry.metricFamilySamples)
    writer.toString
}

val externalRegistry = Task.effect(CollectorRegistry.defaultRegistry)

val program = for {
    r <- externalRegistry
    s <- writeT004(r)
} yield s

val rt = Runtime.unsafeFromLayer(Console.live)

rt.unsafeRun(program >>= (s => putStrLn(s)))


# HELP PrometheusHistogram Sample prometheus histogram
# TYPE PrometheusHistogram histogram
PrometheusHistogram_bucket{class="Users",method="getTotal",le="0.0",} 0.0
PrometheusHistogram_bucket{class="Users",method="getTotal",le="0.2",} 0.0
PrometheusHistogram_bucket{class="Users",method="getTotal",le="0.4",} 1.0
PrometheusHistogram_bucket{class="Users",method="getTotal",le="0.6000000000000001",} 1.0
PrometheusHistogram_bucket{class="Users",method="getTotal",le="0.8",} 1.0
PrometheusHistogram_bucket{class="Users",method="getTotal",le="+Inf",} 1.0
PrometheusHistogram_count{class="Users",method="getTotal",} 1.0
PrometheusHistogram_sum{class="Users",method="getTotal",} 0.32
PrometheusHistogram_bucket{class="Users",method="getUsers",le="0.0",} 0.0
PrometheusHistogram_bucket{class="Users",method="getUsers",le="0.2",} 0.0
PrometheusHistogram_bucket{class="Users",method="getUsers",le="0.4",} 0.0
PrometheusHistogram_bucket{class="Users",method="getUsers",le="0.6000000000000001",} 1.0
Prometheus

import zio.console.putStrLn

import zio.console.Console

import zio.Runtime


writeT004: CollectorRegistry => Task[String] = ammonite.$sess.cmd20$Helper$$Lambda$2970/1100432450@1ff97122
externalRegistry: Task[CollectorRegistry] = zio.ZIO$EffectPartial@389c178d
program: zio.ZIO[Any, Throwable, String] = zio.ZIO$FlatMap@55aea472
rt: Runtime.Managed[Console] = zio.Runtime$Managed$$anon$1@55af49cc

In [22]:
import zio.{ Has, Layer, ZLayer }
import zio.{ RIO, Task }

type Metrics = Has[Metrics.Service]

object Metrics {
    trait Service {
      def getRegistry(): Task[CollectorRegistry]

      def inc(tags: Array[String]): Task[Unit]

      def inc(amount: Double, tags: Array[String]): Task[Unit]

      def time(f: () => Unit, tags: Array[String]): Task[Double]

    }

    val live: Layer[Nothing, Metrics] = ZLayer.succeed(new Service {

      private val (myCounter, myHistogram) = {
          val c = Counter
            .build()
            .name("LayerCounter")
            .labelNames(Array("class", "method"): _*)
            .help(s"Prometheus counter inside a layer")
            .register()

        val h = Histogram
            .build()
            .name("LayerHistogram")
            .labelNames(Array("class", "method"): _*)
            .linearBuckets(0.0, 0.2, 5)
            .help(s"Prometheus histogram inside a layer")
            .register()
          
          (c, h)
      }

      def getRegistry(): Task[CollectorRegistry] =
        Task.effect(CollectorRegistry.defaultRegistry)

      def inc(tags: Array[String]): zio.Task[Unit] =
        inc(1.0, tags)

      def inc(amount: Double, tags: Array[String]): Task[Unit] =
        Task(myCounter.labels(tags:_*)inc(amount))

      def time(f: () => Unit, tags: Array[String]): Task[Double] = Task {
          val t = myHistogram.labels(tags: _*).startTimer()
          f()
          t.observeDuration()
      }
    })
}

import zio.{ Has, Layer, ZLayer }

import zio.{ RIO, Task }


defined type Metrics
defined object Metrics

In [25]:
val program1 = for {
      m  <- RIO.environment[Metrics]
      _  <- putStrLn("Metrics")
      r  <- m.get.getRegistry()
      _  <- m.get.inc(Array("RequestCounter", "get"))
      _  <- m.get.inc(Array("RequestCounter", "post"))
      _  <- m.get.inc(2.0, Array("LoginCounter", "login"))
      _  <- m.get.time(() => Thread.sleep(2000), Array("histogram", "get"))
      s  <- writeT004(r)
      _  <- putStrLn(s)
    } yield ()

program1: zio.ZIO[Metrics with Console, Throwable, Unit] = zio.ZIO$FlatMap@5473d353

In [28]:
CollectorRegistry.defaultRegistry.clear()
rt.unsafeRun(program1.provideSomeLayer[Console](Metrics.live))

Metrics
# HELP LayerHistogram Prometheus histogram inside a layer
# TYPE LayerHistogram histogram
LayerHistogram_bucket{class="histogram",method="get",le="0.0",} 0.0
LayerHistogram_bucket{class="histogram",method="get",le="0.2",} 0.0
LayerHistogram_bucket{class="histogram",method="get",le="0.4",} 0.0
LayerHistogram_bucket{class="histogram",method="get",le="0.6000000000000001",} 0.0
LayerHistogram_bucket{class="histogram",method="get",le="0.8",} 0.0
LayerHistogram_bucket{class="histogram",method="get",le="+Inf",} 1.0
LayerHistogram_count{class="histogram",method="get",} 1.0
LayerHistogram_sum{class="histogram",method="get",} 2.000147973
# HELP LayerCounter Prometheus counter inside a layer
# TYPE LayerCounter counter
LayerCounter{class="RequestCounter",method="get",} 1.0
LayerCounter{class="RequestCounter",method="post",} 1.0
LayerCounter{class="LoginCounter",method="login",} 2.0



In [32]:
type Metrics2 = Has[Metrics2.Service]

object Metrics2 {
    trait Service {
      def getRegistry(): Task[CollectorRegistry]

      def inc(tags: Array[String]): Task[Unit]

      def inc(amount: Double, tags: Array[String]): Task[Unit]

      def time(f: () => Unit, tags: Array[String]): Task[Double]

    }

    val receiver: (Counter, Histogram) => Layer[Nothing, Metrics2] = 
    (myCounter, myHistogram) => ZLayer.succeed(new Service {

      def getRegistry(): Task[CollectorRegistry] =
        Task.effect(CollectorRegistry.defaultRegistry)

      def inc(tags: Array[String]): zio.Task[Unit] =
        inc(1.0, tags)

      def inc(amount: Double, tags: Array[String]): Task[Unit] =
        Task(myCounter.labels(tags:_*)inc(amount))

      def time(f: () => Unit, tags: Array[String]): Task[Double] = Task {
          val t = myHistogram.labels(tags: _*).startTimer()
          f()
          t.observeDuration()
      }
    })
    
    val receiverHas: ZLayer[Has[(Counter, Histogram)], Nothing, Metrics2] =
      ZLayer.fromFunction[Has[(Counter, Histogram)], Metrics2.Service](
        tup => new Service {

            def getRegistry(): Task[CollectorRegistry] =
              Task.effect(CollectorRegistry.defaultRegistry)

            def inc(tags: Array[String]): zio.Task[Unit] =
              inc(1.0, tags)

            def inc(amount: Double, tags: Array[String]): Task[Unit] =
              Task(tup.get._1.labels(tags:_*)inc(amount))

            def time(f: () => Unit, tags: Array[String]): Task[Double] = Task {
              val t = tup.get._2.labels(tags: _*).startTimer()
              f()
              t.observeDuration()
            }
          }
      )
}

defined type Metrics2
defined object Metrics2

In [65]:
CollectorRegistry.defaultRegistry.clear()

In [62]:
val c = Counter
    .build()
    .name("PrometheusCounter")
    .labelNames(Array("class", "method"): _*)
    .help(s"Sample prometheus counter")
    .register()

val h = Histogram
    .build()
    .name("PrometheusHistogram")
    .labelNames(Array("class", "method"): _*)
    .linearBuckets(0.0, 0.2, 5)
    .help(s"Sample prometheus histogram")
    .register()

c: Counter = io.prometheus.client.Counter@49854c92
h: Histogram = io.prometheus.client.Histogram@3e5b19d2

In [59]:
val rLayerHas     = ZLayer.succeed[(Counter, Histogram)]((c, h)) >>> Metrics2.receiverHas
val rtReceiverhas = Runtime.unsafeFromLayer(rLayerHas ++ Console.live)

rLayerHas: ZLayer[Any, Nothing, Metrics2] = zio.ZLayer@45b810bf
rtReceiverhas: Runtime.Managed[Metrics2 with Console] = zio.Runtime$Managed$$anon$1@57180842

In [63]:
val rLayer     = Metrics2.receiver(c, h)
val rtReceiver = Runtime.unsafeFromLayer(rLayer ++ Console.live)

rLayer: Layer[Nothing, Metrics2] = zio.ZLayer@52dff936
rtReceiver: Runtime.Managed[Metrics2 with Console] = zio.Runtime$Managed$$anon$1@4fb42b8b

In [71]:
val program2 = for {
      m  <- RIO.environment[Metrics2]
      _  <- putStrLn("Metrics2")
      _  <- m.get.inc(Array("RequestCounter", "get"))
      _  <- m.get.inc(Array("RequestCounter", "post"))
      _  <- m.get.inc(2.0, Array("LoginCounter", "login"))
      _  <- m.get.time(() => Thread.sleep(2000), Array("histogram", "get"))
      r  <- m.get.getRegistry()
      s  <- writeT004(r)
      _  <- putStrLn(s)
    } yield ()

//rtReceiverhas.unsafeRun(program2)
//rtReceiver.unsafeRun(program2)
rt2.unsafeRun(program2)

Metrics2



program2: zio.ZIO[Metrics2 with Console, Throwable, Unit] = zio.ZIO$FlatMap@2b094571